In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [244]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import csv
import math
import re
import imp
import json
import numpy as np
import pandas as pd
import difflib as dl
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q cltk==1.0.22
%pip install -q strsim
%pip install -q leven

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [211]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import similarities.levenshtein as thesisLevenshteinSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment
import features.model_features as thesisModelFeatures

imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)
imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)
imp.reload(thesisLevenshteinSimilarities)
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

# Create burchard version in original london format

1. create london/zwickau versions without words/letters replacement - replace only numbers, punctuation, etc...
2. create word mapping of a new created versions in 1 to regular processed versions
3. change words in burchard candidate according to mapping in 2 - it should return us burchard in original format

In [212]:
london_corpus_by_new_line_without_words_processing = thesisDataReader.get_london_by_new_line_without_words_processing()
zwickau_corpus_by_new_line_without_words_processing = thesisDataReader.get_zwickau_by_new_line_without_words_processing()

In [213]:
london_original_to_processed_mapping = thesisVocabulary.create_pre_post_processing_map(
    ' \n'.join(london_corpus_by_new_line_without_words_processing)
)

In [214]:
burchard_version_with_original_london_text = []

for p in thesisDataReader.get_burchard_candidate_version_based_on_strongly_similar_london_base():
    new_p = ''
    
    for word in p.split():
        original_word = london_original_to_processed_mapping[london_original_to_processed_mapping['after'] == word]['before'].values[0]
        new_p = f'{new_p} {original_word}' if len(new_p) > 0 else original_word
    
    burchard_version_with_original_london_text.append(new_p)

In [215]:
model_features_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    burchard_version_with_original_london_text
)

In [216]:
model_features_df

,a ad,a aq,a ba,a be,a ca,a ce,a ch,a ci,a co,a cr,...,zrael,zrahe,zucar,zucca,zucur,zurio,zyda,zyph,index,version
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213.0,2.0
852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,214.0,2.0
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.0,2.0
854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,216.0,2.0


In [217]:
X, y = thesisModelFeatures.create_X_y(model_features_df)

In [218]:
y

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
851    2.0
852    2.0
853    2.0
854    2.0
855    2.0
Name: version, Length: 856, dtype: float64

In [219]:
X

,a ad,a aq,a ba,a be,a ca,a ce,a ch,a ci,a co,a cr,...,zotus,zra d,zrael,zrahe,zucar,zucca,zucur,zurio,zyda,zyph
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=101,stratify=y)

In [221]:
X_train

,a ad,a aq,a ba,a be,a ca,a ce,a ch,a ci,a co,a cr,...,zotus,zra d,zrael,zrahe,zucar,zucca,zucur,zurio,zyda,zyph
448,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
827,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
411,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070919,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
615,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
811,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
704,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.058524,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.031199,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
y_train

448    1.0
827    2.0
411    1.0
42     0.0
615    1.0
      ... 
202    0.0
811    2.0
704    2.0
51     0.0
508    1.0
Name: version, Length: 642, dtype: float64

In [262]:
scores_df = pd.DataFrame(dtype=float)
scoring = ('precision_macro', 'recall_macro', 'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy')

In [263]:
svc_cross_validate_score = cross_validate(
    SVC(kernel="linear"),
    X_train,
    y_train,
    cv=10,
#     scoring=('roc_auc', 'average_precision', 'f1'),
    scoring=scoring
#     n_jobs = -1
)

In [266]:
for s in scoring:
    scores_df.loc['SVC_linear', s] = svc_cross_validate_score[f'test_{s}'].mean()

In [267]:
scores_df

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
SVC_linear,0.172213,0.192797,0.176383,0.213558,0.194099,0.213558


In [261]:
svc_cross_validate_score

{'fit_time': array([27.17136192, 27.37078118, 26.40304518, 27.98421693, 29.49357295,
        28.234622  , 29.39745474, 27.34383583, 23.06925201, 27.71009326]),
 'score_time': array([2.29563594, 2.44700694, 2.3078289 , 2.58810711, 2.23552108,
        2.24054503, 2.17283607, 2.21878099, 2.1341238 , 2.90032887]),
 'test_precision_macro': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_recall_macro': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1_macro': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1_micro': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1_weighted': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1_samples': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}

In [233]:
decision_tree_validate_score = cross_validate(
    DecisionTreeClassifier(),
    X_train,
    y_train,
    cv=10,
#     scoring=('roc_auc', 'average_precision', 'f1'),
    scoring=('precision_macro', 'recall_macro', 'f1_macro')
#     n_jobs = -1
)

In [240]:
scores_df.loc['DecisionTreeClassifier', 'precision_macro'] = decision_tree_validate_score['test_precision_macro'].mean()
scores_df.loc['DecisionTreeClassifier', 'recall_macro'] = decision_tree_validate_score['test_recall_macro'].mean()
scores_df.loc['DecisionTreeClassifier', 'f1_macro'] = decision_tree_validate_score['test_f1_macro'].mean()

In [234]:
random_forest_cross_validate = cross_validate(
    RandomForestClassifier(),
    X_train,
    y_train,
    cv=10,
    scoring=('precision_macro', 'recall_macro', 'f1_macro')
)

In [241]:
scores_df.loc['RandomForestClassifier', 'precision_macro'] = random_forest_cross_validate['test_precision_macro'].mean()
scores_df.loc['RandomForestClassifier', 'recall_macro'] = random_forest_cross_validate['test_recall_macro'].mean()
scores_df.loc['RandomForestClassifier', 'f1_macro'] = random_forest_cross_validate['test_f1_macro'].mean()

In [242]:
scores_df

,precision_macro,recall_macro,f1_macro
SVC_linear,0.172213,0.192797,0.176383
DecisionTreeClassifier,0.419308,0.414375,0.414958
RandomForestClassifier,0.246619,0.254249,0.242180


In [243]:
london_corpus_by_new_line_without_words_processing

['liber de terra sancta',
 'cum in veteribus hystoriis legamus sicut dicit beatus ieronimus quosdam lustrasse provincias maria transfretasse ut ea que ex libris noverant coram positi viderent ut plato menphiticos vates et egyptum apollonius qui persas intravit transivit caucasum albanos scithas massagethas indiam bragmanos quoque ut yartham videret et tandem egyptum intravit ut famosam mensam solis videret in sabulo quid mirum si christiani terram illam quam christi sonant ecclesie universe videre et visitare desiderant vanerabantur antiqui sancta sanctorum quia ibi erat archa testamenti et cherubin cum propiciatorio et manna et virga aaron que fronduerat que omnia erant umbra futuri nonne aput nos venerabilis est sepulchrum dulcis ihesu quod quociens quis ingreditur tociens involutum syndone mentis videt oculis salvatorem et paululum procedens videt lapidem revolutum angelum in eo sedentem et sudarium cum lintheaminibus mulieribus ostendentem',
 'quis christianus hiis visis non festin

In [245]:
gaussian_nb_cross_validate = cross_validate(
    GaussianNB(),
    X_train,
    y_train,
    cv=10,
    scoring=('precision_macro', 'recall_macro', 'f1_macro')
)

In [246]:
scores_df.loc['GaussianNB', 'precision_macro'] = gaussian_nb_cross_validate['test_precision_macro'].mean()
scores_df.loc['GaussianNB', 'recall_macro'] = gaussian_nb_cross_validate['test_recall_macro'].mean()
scores_df.loc['GaussianNB', 'f1_macro'] = gaussian_nb_cross_validate['test_f1_macro'].mean()

In [247]:
scores_df

,precision_macro,recall_macro,f1_macro
SVC_linear,0.172213,0.192797,0.176383
DecisionTreeClassifier,0.419308,0.414375,0.414958
RandomForestClassifier,0.246619,0.254249,0.242180
GaussianNB,0.110943,0.136841,0.121255


In [249]:
resp = thesisCosineSimilarities.get_inner_version_all_similarities(london_corpus_by_new_line_without_words_processing)

In [254]:
# add features to each p: mean cosine similarity, number of 0 similarities(make sure it is scalled)
resp['5_gram'][0]

[(19, 0.10185792396234644),
 (21, 0.09181683075469789),
 (265, 0.09104424053571192),
 (271, 0.08883917740239196),
 (316, 0.08485746263723083),
 (46, 0.07970776625615915),
 (208, 0.07801358803500882),
 (261, 0.07800920236913238),
 (310, 0.07420364008329801),
 (131, 0.05634888916697893),
 (213, 0.05322256417559036),
 (71, 0.050994140900150656),
 (120, 0.050151305376801736),
 (183, 0.04611609558372975),
 (307, 0.0451533355890884),
 (115, 0.04443682482367232),
 (8, 0.0433766410746058),
 (92, 0.03965644573373061),
 (117, 0.03855291644985953),
 (20, 0.03597091980923719),
 (5, 0.035624886614566045),
 (198, 0.03481835361263374),
 (306, 0.034769829712010404),
 (160, 0.03331304516234357),
 (184, 0.032338284721656035),
 (83, 0.030600982512455275),
 (40, 0.030008394162349197),
 (273, 0.029522269589735254),
 (291, 0.02937708266699418),
 (28, 0.029274069918735646),
 (31, 0.028927482703301817),
 (53, 0.027771893512286268),
 (143, 0.027654925804287096),
 (114, 0.027304285905211143),
 (1, 0.02677236394